# Garlic model

Synthesis: Development and application of integrative plant models - A case study on garlic (*Allium sativum* L.)

## Objectives {.unnumbered}
1. Examine the garlic model example that integrates underlying lower-level processes as sub-modules. 
2. Experiment with applying integrative models to address research questions and real world problems.

## Readings {.unnumbered}

-   (required) Hsiao J, Yun K, Moon KH, Kim S-H (2019) A process-based model for leaf development and growth in hardneck garlic (*Allium sativum*). Annals of Botany 124(7), 1143-1160 (<https://doi.org/10.1093/aob/mcz060>)

-   (recommended) Yun K, Shin M, Moon KH, Kim S-H (2022) An integrative process-based model for biomass and yield estimation of hardneck garlic (*Allium sativum*). Frontiers in Plant Science 13, 783810 (<https://doi.org/10.3389/fpls.2022.783810>)


## Questions {.unnumbered}

1. Have you grown garlic? What are interesting features and attributes (i.e., traits) about garlic as a plant and crop?
2. When will be the best time to plant garlic in Seattle?
3. Will the optimal planting time shift with rising CO2 and increasing temperatures?

## Exercises {.unnumbered}

We examine a process-based model for garlic by Hsiao et al. (2019) in this exercise. This model integrates
most of the plant processes we discussed in this class based on principles and methods we addressed in this course.

1. Inspect the model structure.
2. Identify plant processes represented in the model.
3. Identify state variables, parameters, and environmental variables of the model.
4. Run simulations using the weather and crop information available in Garlic package.
5. Perform sensitivity analyses of model parameters on key state variables.
6. Apply the model to assess climate change impact on a southern cultivar of garlic: Namdo in Jeju, Korea where they are currently grown.
7. Apply the model to evaluate climate adaptation options such as changing planting dates and moving cultivation to a northern location: Chuncheon, Korea.
8. Add interpretation and discussion to your responses to all questions as appropriate.

In [ ]:
using Cropbox

In [ ]:
#| echo: false
#| output: false
Cropbox.Interact.WebIO.setup(:ijulia);

In [ ]:
using CSV
using DataFrames
using Dates
import Gadfly as G
using TimeZones

## Model overview

We examine a process-based model for garlic by [Hsiao et al. (2019)](https://doi.org/10.1093/aob/mcz060) and [Yun et al. (2022)](https://doi.org/10.3389/fpls.2022.783810) in this exercise. This model integrates most of the plant processes we’ve discussed so far in the class.

![Garlic model overview](./figs/garlic_model.png){#fig-garlic-model width="100%"}

## Architecture

#### Exercise: Inspect the model structure.

The [original model](https://doi.org/10.1093/aob/mcz060) was written in C++ and later [reimplemented](https://doi.org/10.3389/fpls.2022.783810) in Julia as a testbed for developing Cropbox framework.

In [ ]:
using Garlic

The garlic model is named `Model` system in the `Garlic` package.

In [ ]:
Cropbox.hierarchy(Garlic.Model; skipcontext = true)

While the model may look quite complex, the structure of the model shouldn't be too unfamiliar since the most parts of underlying concepts have been already covered throughout the quarter. For example, you can notice `GasExchange` system which is almost the same as the coupled gas-exchange model we covered in **Lab06**. `Phenology` system for tracking phenology consists of `BetaFunction` and `GrowingDegree` mixins we covered in **Lab07**. `Leaf` system for modeling leaf growth is not as simple as simple growth models we covered in **Lab02** and **Lab04**, but relying on the same principle.

If interested in the details how the model is implemented, feel free to read source code in the [Garlic.jl](https://github.com/cropbox/Garlic.jl) repository.

## Processes

#### Exercise: Identify plant processes represented in the model.

### Phenology

Let's take an in-depth look at the phenology (`Phenology`) part. There are systems representing each developmental stage of plant such as germination (`Germination`), canopy emergence (`Emergence`), leaf appearance (`LeafAppearance`), and so on. Many of them are driven by thermal time accumulation based on `BetaFunction` mixin, which is indicated by outgoing solid edges in the hierarchy diagram below.

In [ ]:
Cropbox.hierarchy(Garlic.Phenology; skipcontext = true)

Here, we choose `LeafAppearance` system to see its imeplementation, especially in comparison to phenology models we created last week. Cropbox provides `source()` function for inspecting model specification of an existing system.

In [ ]:
Cropbox.source(Garlic.LeafAppearance)

Well, it's not too legible due to autogenearated comments and no highligthing. Let's write it down by hand below. Some extra variables are added to the model we created last week, but the essentital concept remains the same. The leaf appearance rate per unit time (`r*β`) calculated using Beta function accumulates to `LTA` which is then added to the initial leaf count (`ILTA` = 1) to form the current number of leaves have appeared (`leaves_appeared`).

```julia
@system LeafAppearance(Stage, Emergence, LeafInitiation) begin
    ..

    LTA(r = LTAR_max, β = BF.ΔT): leaf_tip_appearance => r*β ~ accumulate(when = leaf_appearing)

    ILTA: initial_leaf_tip_appearance => 1            ~ track::int(when = emerged)
    leaves_appeared(ILTA, LTA)        => (ILTA + LTA) ~ track::int(round = :floor)
    
    ..
end
```

Then, how is the rate of Beta function `r`, that is the maximum leaf appearance rate (`LTAR_max`), determined? It used to be a parameter with a constant value in the previous model, but here we want to make it a little bit more advanced. We observed through field experiments that the stroage conditions (*i.e.* duration, temperature) of garlic seed bulbs could influence the leaf appearance rate once they were planted. Those seed bulbs with a longer storage term often ended up having leaves appeared much faster than the others. For modeling this process, we modified `LTAR_max` from a constant parameter to a dynamic variable adjusted by a logisitic function.

Aside from technical details, just note that the variable originally declared as `preserve(parameter)` now became `track` to accomodate a new process. Some related variables (*e.g.* `LTAR_max0`) and parameters (*e.g.* `LTARa_max`, `_SDm`, `_k`) were also added, but the overall structure of the model system remains largely the same.

```julia
@system LeafAppearance(Stage, Emergence, LeafInitiation) begin
    LTARa_max: maximum_phyllochron_asymptote => 0.4421   ~ preserve(parameter, u"d^-1")
    _SDm                                     => 117.7523 ~ preserve(parameter, u"d")
    _k                                       => 0.0256   ~ preserve(parameter, u"d^-1")

    LTAR_max0(LTARa_max, SD, _SDm, _k): initial_maximum_phyllochron => begin
        LTARa_max / (1 + exp(-k * (SD - SDm)))
    end ~ preserve(u"d^-1", parameter)

    LTAR_max(LTAR_max0, LTARa_max, n=leaves_appeared, ng=leaves_generic): maximum_phyllochron => begin
        n0 = 0
        r0 = LTAR_max0
        r1 = LTARa_max / 2
        r0 + (r1 - r0) * (clamp(n, n0, ng) - n0) / (ng - n0)
    end ~ track(u"d^-1")

    ..
end
```

The remaining variables mostly include `flag` variables such as `leaf_appearable` for checking if more leaves are ready to appear, `leaf_appeared` for checking if all leaves have appeared, and `leaf_appearing` for indicating more leaves are about to appear. These conditional variables are especially useful for keeping track of developmental stages in other parts of the phenolofy models to start/stop certain processes. In the example above, we can find some states are shared from other stages like canopy emergence (`emerged`) and leaf initiation (`leaves_initiated`). These external variables are accessible in `LeafAppearance` system because it contains `Emergence` and `LeafInitiation` as mixins.

```julia
@system LeafAppearance(Stage, Emergence, LeafInitiation) begin
    ..
    
    leaf_appearable(emerged) ~ flag
    leaf_appeared(leaves_appeared, leaves_initiated) => begin
        0 < leaves_initiated <= leaves_appeared
    end ~ flag
    leaf_appearing(leaf_appearable & !leaf_appeared) ~ flag
end
```

Now let's take out the phenology module as a standalone system for testing. Some shared dependencies like `Weather` have to be manually created.

In [ ]:
@system PhenologyController(Garlic.Phenology, Controller) begin
    calendar(context)               ~ ::Calendar

    weather(context, calendar)      ~ ::Garlic.Weather
    sun(context, calendar, weather) ~ ::Garlic.Sun
    soil(context)                   ~ ::Garlic.Soil
end

Also create a configuration using the dataset included in `Garlic.jl` package. The weather data comes from the year of 2010 collected from the site (ID = 185) called Gosan located at Jeju Island in South Korea. Planting date was November 1st.

In [ ]:
pheno_config = @config(
    :Calendar  => :init          => ZonedDateTime(2010, 10, 1, tz"Asia/Seoul"),
    :Weather   => :store         => Garlic.loadwea(Garlic.datapath("Korea/185_2010.wea"), tz"Asia/Seoul"),
    :Phenology => :planting_date => ZonedDateTime(2010, 11, 1, tz"Asia/Seoul"),
)

Now we can plot the number of leaves appeared (`leaves_appeared`) along with the number of leaves initiated (`leaves_initiated`) simulated for about 10 months (300 days).

In [ ]:
visualize(PhenologyController, "calendar.date", [:leaves_appeared, :leaves_initiated];
    config = pheno_config,
    stop = 300u"d",
    snap = 1u"d",
    title = "Leaf Count",
    names = ["Appeared", "Initiated"],
    kind = :line,
)

As we rely on Beta function, the interval between two leaves can be sometimes shorter, sometimes longer. Notably, the interval from January to February looks much longer and you can figure out the reason by looking at the temperature (`T`) plot.

In [ ]:
visualize(PhenologyController, "calendar.date", :T;
    config = pheno_config,
    stop = 300u"d",
    snap = 1u"d",
    kind = :line,
)

The garlic model also internally keeps track of major developmental stages with `development_phase` variable declared in `Symbol` type. The vegeatative stage (`vegetative`) began after planting in early November and remained until early April when leaf initiation stopped. All those leaves initiated during the vegetative stage would still continue growing and appear by mid May as we found in the previous plot. At this point, the plant development was already in the reproductive stage (`bulb_growth_before_scape_appearance`). The scape appeared in early June and its removal could be controlled via configuration, resulting into changes in carbon partitioning dynamics.

In [ ]:
visualize(PhenologyController, "calendar.date", :development_phase;
    config = pheno_config,
    stop = 300u"d",
    snap = 1u"d",
    kind = :step,
)[] |> G.SVG(8G.inch, 3G.inch)

Multiple stages of phenological development found in crop models may seemingly complex at first, then you would soon realize that many of them actually rely on the simple concept of thermal time accumulation. Perhaps it'd be more important to think about how to link multiple pieces into a single model.

### Photosynthesis

We already covered the structure of leaf-level coupled leaf-gas exchange model and its scaling to canopy-level in **Lab06**. `Garlic.jl` includes the almost same impelementation for calculating the total amount of carbon assimilation. `GasExchange` system implements the leaf-level gas-exchange and `Photosynthesis` system scales it up to the canopy-level.

In [ ]:
Cropbox.hierarchy(Garlic.Photosynthesis; skipcontext = true)

Remember that canopy-level scaling up was done in terms of the two-leaves approach via `Radiation` system for separating sunlit and shaded leaves. More details can be found on [De Pury and Farquhar (1997)](https://doi.org/10.1111/j.1365-3040.1997.00094.x). In our garlic model, we have two separate `GasExchange` systems for the two types of leaf and `Radiation` system calculates irradiance (`Q_sun`, `Q_sh`) and leaf area index (`LAI_sunlit`, `LAI_shaded`) for them. The total net photosynthesis rate at the canopy level (`A_net`) is determined by summing up two `A_net` from each gas-exchange systems in proportion to their respective `LAI`.

```julia
@system Photosynthesis begin
    ..
   
    radiation(context, sun, leaf_area_index) ~ ::Radiation

    sunlit_gasexchange(context, soil, weather, PPFD = Q_sun, LAI = LAI_sunlit) ~ ::GasExchange
    shaded_gasexchange(context, soil, weather, PPFD = Q_sh,  LAI = LAI_shaded) ~ ::GasExchange

    LAI_sunlit(radiation.sunlit_leaf_area_index): sunlit_leaf_area_index ~ track
    LAI_shaded(radiation.shaded_leaf_area_index): shaded_leaf_area_index ~ track

    Q_sun(radiation.irradiance_Q_sunlit): sunlit_irradiance ~ track(u"μmol/m^2/s")
    Q_sh(radiation.irradiance_Q_shaded):  shaded_irradiance ~ track(u"μmol/m^2/s")

    A_net(
        a = sunlit_gasexchange.A_net_total,
        b = shaded_gasexchange.A_net_total,
    ): net_CO2_umol_per_m2_s => a + b ~ track(u"μmol/m^2/s")

    ..
end
```

Let's make `Photosynthesis` as a standalone system for isolated testing.

In [ ]:
@system PhotosynthesisController(Garlic.Photosynthesis, Garlic.Weight, Garlic.Water, Controller) begin
    calendar(context)                ~ ::Calendar

    weather(context, calendar)       ~ ::Garlic.Weather
    sun(context, calendar, weather)  ~ ::Garlic.Sun
    soil(context)                    ~ ::Garlic.Soil

    LAI(LAI0): leaf_area_index       ~ track(u"m^2/m^2")
    LAI0:      leaf_area_index_param ~ preserve(parameter, u"m^2/m^2")
    PD:        plant_density         ~ preserve(parameter, u"m^-2")
end

We basically use the same dataset as we used for phenology. Here we add parameters specific to photosynthesis such as leaf area index (`LAI`) and plant density (`PD`).

In [ ]:
photo_config = @config(
    :Calendar                 => :init  => ZonedDateTime(2010, 10, 1, tz"Asia/Seoul"),
    :Weather                  => :store => Garlic.loadwea(Garlic.datapath("Korea/185_2010.wea"), tz"Asia/Seoul"),
    :PhotosynthesisController => (:LAI0 => 5, :PD => 55),
)

First, plot the net photosynthesis rate (`A_net`). It's not per leaf but per unit land area. By the way, even though we ran the model from October 1st, the actual result was mostly driven by `LAI` and `PD` parameters we supplied. We're just testing a subset of the model, not the full garlic model here.

In [ ]:
visualize(PhotosynthesisController, "calendar.time", :A_net;
    config = photo_config,
    stop = 30u"d",
    title = "Net photosynthesis rate (A_net)",
    kind = :line,
)   

Here are the net photosynthesis rates for sunlit and shaded leaves separated.

In [ ]:
visualize(PhotosynthesisController, "calendar.time", [
        "sunlit_gasexchange.A_net_total",
        "shaded_gasexchange.A_net_total",
    ];
    config = photo_config,
    stop = 30u"d",
    title = "Net photosynthesis rate (A_net)",
    names = ["Sunlit", "Shaded"],
    kind = :line,
)   

Running the same simulation in a short period (3 days) shows diurnal patterns more clearly.

In [ ]:
visualize(PhotosynthesisController, "calendar.time", [
        :A_net,
        "sunlit_gasexchange.A_net_total",
        "shaded_gasexchange.A_net_total",
    ];
    config = photo_config,
    stop = 3u"d",
    title = "Net photosynthesis rate (A_net)",
    names = ["Total", "Sunlit", "Shaded"],
    kind = :line,
)   

We can also split out LAI for the two leaves. Here we use the fixed value of 5 for the total LAI and the ratio between sunlit and shaded leaves dynamically change depending on the position of sun.

In [ ]:
visualize(PhotosynthesisController, "calendar.time", [
        "radiation.LAI_sunlit",
        "radiation.LAI_shaded",
    ];
    config = photo_config,
    stop = 300u"d",
    title = "Leaf Area Index (LAI)",
    names = ["Sunlit", "Shaded"],
    kind = :line,
)   

For instance, in winter time, *e.g.* January 1st, the elevation of sun goes down and the ratio of direct sunlight reaching the sunlit leaf becomes relatively smaller.

In [ ]:
visualize(PhotosynthesisController, "calendar.time", [
        "radiation.LAI_sunlit",
        "radiation.LAI_shaded",
    ];
    config = (photo_config, :Calendar => :init  => ZonedDateTime(2011, 1, 1, tz"Asia/Seoul")),
    stop = 3u"d",
    title = "Leaf Area Index (LAI)",
    names = ["Sunlit", "Shaded"],
    kind = :line,
)   

Compare with summer time, *e.g.* July 1st, and see how the portion of sunlit leaf increases. The ratio also changes diurnally as sun rises and sets.

In [ ]:
visualize(PhotosynthesisController, "calendar.time", [
        "radiation.LAI_sunlit",
        "radiation.LAI_shaded",
    ];
    config = (photo_config, :Calendar => :init  => ZonedDateTime(2011, 7, 1, tz"Asia/Seoul")),
    stop = 3u"d",
    title = "Leaf Area Index (LAI)",
    names = ["Sunlit", "Shaded"],
    kind = :line,
)   

We can also plot the amount of light received by the sunlit and shaded leaves. These are the irradiance values fed into two independent `GasExchange` instances.

In [ ]:
visualize(PhotosynthesisController, "calendar.time", ["radiation.sun.PARtot", :Q_sun, :Q_sh];
    config = photo_config,
    stop = 3u"d",
    title = "Irradiance",
    names = ["Total", "Sunlit", "Shaded"],
    kind = :line,
)   

### Biomass

#### Growth Rate

Do you remember we started modeling with simple growth functions like logistic and Gompertz in **Lab02**? At the time, we were trying to capture the shape of growth in a single curve, but now we know that the actual growth in terms of biomass isn't just the response of a single function, but rather shown up as an *emergent property* arising from multiple processes.

In [ ]:
sim = simulate(Garlic.Model;
    config = Garlic.Examples.RCP.ND_RICCA_2014_field,
    stop = "calendar.count",
);

Let's confirm this by running the garlic model. We use the dataset collected by Research Institute of Climate Change and Agriculture (RICCA) located at South Korea in 2014. The parameters were specifically calibrated for the local cultivar named Namdo (ND).

In [ ]:
visualize(sim, :time, :total_mass; kind = :line)

This is the plot of total biomass simulated by the model for a season. It somewhat looks like the first half of an exponential curve or a smooth line generated by a Gompertz function. But as you know, this is not a curve directly generated by a single process.

The total biomass is actually a sum of biomass partitioned into individual organs. Among them, the biomass partitioned to the bulb decides the *yield* of garlics.

In [ ]:
visualize(sim, :time, [:total_mass, :living_leaf_mass, :bulb_mass, :living_stalk_mass, :visible_scape_mass];
    kind  = :line,
    names = ["Total", "Leaf", "Bulb", "Stalk", "Scape"],
)

There are many factors influencing the calculation of total biomass and the total carbon assimilation rate per unit time (`total_carbon`) is one of them. This value is determined by the gross photosynthesis rate (`A_gross`) scaled up to a unit land area, as we saw in the gas-exchange part, and then adjusted for an individual canopy after taking account of respiration and conversion efficiency. For a single plant, this is the amount of change (increase or decrease) in biomass per unit time and the amount to be partitioned into multiple organs.

In [ ]:
visualize(sim, :time, :total_carbon; kind = :line)

In **Lab02**, we first started with a constant growth rate, *e.g.* 0.03, for an exponential growth model, then soon realized that the growth rate should somehow decrease over time. In our garlic model, we don't have a specific parameter representing the growth rate, but instead we calculate one. If we divide the change of carbon at a specific time by the total biomass at the time, the result is the growth rate (`r`) that we discussed.

In [ ]:
visualize(sim, :time, :(total_carbon / total_mass);
    kind = :line,
    ylab = "Growth rate",
    name = "",
    ylim = (-0.01, 0.2),
)

As expected, the growth rate has an overall decreasing pattern as a result of interaction between multiple processes responding to environmental conditions and developmental stages. Such discovering of an emergent property is one of primary reasons we want to make and use process-based models in our research.

#### Leaf Growth

The total amount of available carbon per unit time (`total_carbon`) we discussed above is determined by the carbon assimilation rate (`total_supply`).

In [ ]:
visualize(sim, :time, :carbon_supply; kind = :line)

And the carbon assimilation rate (`total_supply`) is directly related to the photosynthesis rate (`A_gross`).

In [ ]:
visualize(sim, :time, :A_gross; kind = :line)

And we need leaf area index (LAI) to convert the photosynthesis rate (`A_gross`) per leaf area to the photosynthesis rate per a unit land area. The shape of leaf area index actually looks quite similar to the photosynthesis rate above.

In [ ]:
visualize(sim, :time, :LAI;
    title = "Leaf Area Index",
    name = "",
    kind = :line,
)

The leaf area index (LAI) is defined as the total leaf area divided by the land area. As the land area is fixed, the actual shape of LAI follows the total leaf area.

In [ ]:
visualize(sim, :time, :green_leaf_area;
    title = "Total Green Leaf Area",
    name = "",
    kind = :line,
)

Then how we can calculate the total leaf area? There are many different approaches adopted by crop models. Many crop models, especially those relying on the concept of radiation use efficiency (RUE) instead of a more complicated gas-exchange model, often do not simulate the leaf area expansion directly, but rather derive it by multiplying biomass with a constant parameter of specific leaf area (SLA), which is defined as the total leaf area divided by total biomass. In our garlic model, however, we opted to model leaf growth as a separate process and simulate leaf expansion at an individual leaf-level.

In [ ]:
plot_leaves(v;
    config = Garlic.Examples.RCP.ND_RICCA_2014_field,
    index = :time,
    target = [],
    stop = "calendar.count",
    snap = 1u"d",
    legend = "Rank",
    kw...
) = begin
    r = simulate(Garlic.Model;
        config, index, target, stop, snap,
        snatch = (D, s) -> begin
            for nu in s.NU
                D[end][Symbol(nu.rank')] = nu.leaf[v]'
            end
        end
    )
    n = ncol(r) - 1 - length(target)
    visualize(r, index, string.(1:n); legend, kw...)
end

Let's take a look at how leaf area for individual leaves change over time. Each leaf (`Leaf`) enclosed in a nodal unit (`NodalUnit`) has its own developmental schedule determined by a shared phenology module. Leaf position (rank) also determines the total length of each leaf, which is eventually converted to the leaf area. The leaf area of first leaf can be accessed by `NU[1].leaf.area` variable from the model. `plot_leaves()` function collects such a variable from each leaf to make a plot.

In [ ]:
plot_leaves(:green_area;
    title = "Green Leaf Area",
    kind = :line,
    ylim = (0, 150),
)

We can see 16 leaves in total grow in their own timing and vigor. They go through leaf area expansion, maturation, and senescence. The rank of leaf starts from 1 for the initially appeared leaf and counts on. Overall, leaves appeared later tend to grow larger and their stay green period dynamically changes depending on the time.

Leaf area is allometrically derived from leaf elongation, which is implemented as a thermal time accumulation using Beta function.

In [ ]:
plot_leaves(:length;
    title = "Leaf Length",
    kind = :line,
    ylim = (0, 100),
)

The potential length of each leaf is determined by the total number of leaves at the time of initiation and the position (rank) of the newly formed leaf. You may refer to [Hsiao *et al.* 2019](https://doi.org/10.1093/aob/mcz060) for more details.

In [ ]:
plot_leaves(:potential_length;
    title = "Potential Length",
    kind = :line,
    ylim = (0, 100),
)

The overall magnitude of potential length is primarily controlled by a parameter called the minimum length of longest leaf (`LM_min`). Let's take a look at how it affects leaf area expansion in comparison to the actual observation dataset. This is a simple example of sensitivity analysis.

In [ ]:
obs = CSV.read(Garlic.datapath("Korea/ricca_2014_field.csv"), DataFrame) |> unitfy;

In [ ]:
p = visualize(obs, :measuring_date, :leaf_area;
    xlim = (DateTime(2014, 10, 1), DateTime(2015, 6, 30)),
    ylim = (0, 1200),
)
visualize!(p, Garlic.Model, :time, :green_leaf_area;
    config = Garlic.Examples.RCP.ND_RICCA_2014_field,
    stop = "calendar.count",
    snap = 1u"d",
    group = :Leaf => :LM_min => [60, 70, 80, 90, 100],
    kind = :line,
)

## Parameters

#### Exercise: Identify state variables, parameters, and environmental variables of the model.

While we already saw how the model works above, let's give it a look again. Most of these parameters are described in [Hsiao *et al.* (2019)](https://doi.org/10.1093/aob/mcz060).

In [ ]:
parameters(Garlic.Model; alias = true, recursive = true)

`Garlic` package also provides a few sample configurations in its `Examples` module for testing. We'll pick up `RCP.ND_RICCA_2014_field` which is a configuration for the experiment conducted in 2014 with a cultivar named Namdo (ND) in the field located at the Research Institute of Climate Change and Agriculture (RICCA) in Jeju, Korea.

In [ ]:
config = Garlic.Examples.RCP.ND_RICCA_2014_field

## Simulations

#### Exercise: Run simulations using the weather and crop information available in Garlic package.

We'll rely on `simulate()` function as we always did, but with a slightly different syntax. Let's take a look at the arguments one by one.

The number of model updates for stop condition (`stop`) is provided by `count` variable in the `calendar` system and is determined by start (`calendar.init`) and end date (`calendar.last`) specified in the configuration.

In [ ]:
stop = "calendar.count"

For target variables (`target`) included in the output, not only all variables exposed in the `Model` system (`"*"`), we also want to extract other variables from the nested phenology system (`"pheno.*"`).

In [ ]:
target = ["*", "pheno.*"]

`snap` determines how often a *snapshot* of the model should be captured and recorded in the output. Since the default time unit (`1u"hr"`) is too fine-grained in the scale of garlic growth which takes several months, we want to write down our results in a daily unit (`1u"d"`) instead for performance reason.

In [ ]:
snap = 1u"d"

After setting up all required arguments, running simulation is simple as `simulate()` for other models. It will just take a while to run for the first time as we already got used to do with how Julia works.

In [ ]:
sim = simulate(Garlic.Model; config, stop, target, snap)

### Output

The result contains lots of variables. Here is a graph of simulated green leaf area over time.

In [ ]:
visualize(sim, :time, :green_leaf_area; kind = :line)

For comparison, we'll load up an observation dataset included in the package.

In [ ]:
using CSV
using DataFrames

obs = CSV.read(Garlic.datapath("Korea/ricca_2014_field.csv"), DataFrame) |> unitfy

Here is a plot of the green leaf area with observation.

In [ ]:
p = visualize(sim, :time, :green_leaf_area;
    kind = :line,
    ylim = (0, 1500),
    xlab = "Time",
    ylab = "Leaf area",
    name = "",
)
visualize!(p, obs, :measuring_date, :leaf_area;
    name  = "",
    color = 1,
)

Similary, we can make other plots. Here is a plot of biomass accumulated for each organ.

In [ ]:
p = visualize(sim, :time, [:living_leaf_mass, :bulb_mass, :living_stalk_mass, :visible_scape_mass];
    kind  = :line,
    ylim  = (0, 30),
    names = ["Leaf", "Bulb", "Stalk", "Scape"],
)
visualize!(p, obs, :measuring_date, [:leaf_dry_weight, :bulb_dry_weight, :stem_dry_weight, :scape_dry_weight];
    colors = [1, 2, 3, 4],
    names  = [],
)

Here is a plot of leaf count per leaf development stage with observation.

In [ ]:
p = visualize(sim, :time, :leaves_initiated;
    kind = :line,
    ylim = (0, 16),
    xlab = "Time",
    ylab = "Leaf count",
    name = "Initiated",
)
visualize!(p, sim, :time, [:leaves_appeared, :leaves_dropped, :leaves_fresh];
    kind  = :line,
    names = ["Appeared", "Dropped", "Total"],
)
visualize!(p, obs, :measuring_date, :leaf_count;
    name  = "",
    color = 4,
)

## Sensitivity Analysis

#### Exercise: Perform sensitivity analyses of model parameters on key state variables.

We can conduct a simple sentivity analysis by running the model for multiple times with a slightly different value of parameter.

In [ ]:
using Dates

xlim = (DateTime(2014,10,1), DateTime(2015,6,30))

### Leaf traits

#### Leaf Tip Appearance Rate (asymptote)

In [ ]:
config[:Phenology][:maximum_phyllochron_asymptote]

In [ ]:
p = visualize(obs, :measuring_date, :leaf_area;
    xlim,
    ylim = (0,1500),
)
visualize!(p, Garlic.Model, :time, :green_leaf_area;
    config, stop, snap,
    group = :Phenology => :LTARa_max => [0.35, 0.4, 0.45, 0.5, 0.55],
    kind  = :line,
)

#### Initial Leaves at Harvest

In [ ]:
config[:Phenology][:initial_leaves_at_harvest]

In [ ]:
p = visualize(obs, :measuring_date, :leaf_area;
    xlim,
    ylim = (0,1500),
)
visualize!(p, Garlic.Model, :time, :green_leaf_area;
    config, stop, snap,
    group = :Phenology => :ILN => [3, 4, 5, 6],
    kind  = :line,
)

#### Leaf Initiation Rate

In [ ]:
config[:Phenology][:maximum_leaf_initiation_rate]

In [ ]:
p = visualize(obs, :measuring_date, :leaf_area;
    xlim,
    ylim = (0,1500),
)
visualize!(p, Garlic.Model, :time, :green_leaf_area;
    config, stop, snap,
    group = :Phenology => :LIR_max => [0.05, 0.1, 0.15],
    kind  = :line,
)

#### Minimum Length of Longest Leaf

In [ ]:
config[:Leaf][:minimum_length_of_longest_leaf]

In [ ]:
p = visualize(obs, :measuring_date, :leaf_area;
    xlim,
    ylim = (0,1500),
)
visualize!(p, Garlic.Model, :time, :green_leaf_area;
    config, stop, snap,
    group = :Leaf => :LM_min => [60, 70, 80, 90],
    kind  = :line,
)

#### Leaf Elongation Rate

In [ ]:
config[:Leaf][:maximum_elongation_rate]

In [ ]:
p = visualize(obs, :measuring_date, :leaf_area;
    xlim,
    ylim = (0,1500),
)
visualize!(p, Garlic.Model, :time, :green_leaf_area;
    config, stop, snap,
    group = :Leaf => :LER_max => [3.5, 4, 4.5, 5, 5.5],
    kind  = :line,
)

### Phenology traits

#### Critical Photoperiod

In [ ]:
config[:Phenology][:critical_photoperiod]

In [ ]:
p = visualize(obs, :measuring_date, :leaf_area;
    xlim,
    ylim = (0,1500),
)
visualize!(p, Garlic.Model, :time, :green_leaf_area;
    config, stop, snap,
    group = :Phenology => :critPPD => [10, 10.5, 11, 11.5, 12, 12.5, 13],
    kind  = :line,
)

In [ ]:
visualize(Garlic.Model, :time, "pheno.sun.day_length";
    config, stop, snap,
    kind = :line,
)

#### Stay Green

In [ ]:
config[:Leaf][:stay_green]

In [ ]:
p = visualize(obs, :measuring_date, :leaf_area;
    xlim,
    ylim = (0,1500),
)
visualize!(p, Garlic.Model, :time, :green_leaf_area;
    config, stop, snap,
    group = :Leaf => :SG => [1.2, 1.4, 1.6, 1.8],
    kind  = :line,
)

### Management practices

#### Storage Temperature

In [ ]:
config[:Phenology][:storage_temperature]

In [ ]:
p = visualize(obs, :measuring_date, :leaf_area;
    xlim,
    ylim = (0,1500),
)
visualize!(p, Garlic.Model, :time, :green_leaf_area;
    config, stop, snap,
    group = :Phenology => :ST => [3, 4, 5, 6, 7],
    kind  = :line,
)

#### Planting Density

In [ ]:
config[:Plant][:initial_planting_density]

In [ ]:
visualize(Garlic.Model, :time, :fresh_yield;
    config, stop, snap, xlim,
    group = :Density => :PD0 => [25, 35, 45, 55, 65],
    ylim  = (0,10000),
    kind  = :line,
)

## Application

#### Exercise: Apply the model to assess climate change impact on a southern cultivar of garlic: Namdo in Jeju, Korea where they are currently grown.

Although the garlic model presented above is not as simple as other models we studied in the class, the Cropbox interface we use for interacting with the model remains mostly unchanged. Hope this gives you an idea how a large model can be composed of smaller pieces and used in the research.

Below is an application of the garlic model we've done recently. We used the model for testing a simple climate adaptation strategy, such as plating date shift, under climate change scenarios. More information is available with [Yun et al. (2022)](https://doi.org/10.3389/fpls.2022.783810).

Note that actual code for doing **Ex 1.6** and **Ex 1.7** can be found at [GitHub](
https://github.com/cropbox/Garlic.jl/blob/main/src/examples/rcp.jl) as it gets a bit complex to run a large number of simulations and produce a collective plot out of them. We only provide the final results here.

### Geography

Here are two locations in Korea. Gosan in Jeju island has warm climate that allows growing tons of garlic commercially. Chuncheon has cold winter not viable for growing this particular type of cultivar (Namdo) under current climate.

<img src="https://www.biorxiv.org/content/biorxiv/early/2021/11/04/2021.11.04.467009/F2.large.jpg" width="300">

### Future yield predictions

#### Yield Projection (Gosan)

Here is the yield projection with a range of planting dates under current and future climate projection in Gosan, Jeju which is a region grows ND cultivar at commercial scale.

![](https://www.biorxiv.org/content/biorxiv/early/2021/11/04/2021.11.04.467009/F8.large.jpg)

####  Exercise: Apply the model to evaluate climate adaptation options such as changing planting dates and moving cultivation to a northern location: Chuncheon, Korea.

#### Yield Projection (Chuncheon)

Here is the yield projection with a range of planting dates under current and future climate projection in Chuncheon, Gangwon where current climate is not favorable for growing ND cultivar.

![](https://www.biorxiv.org/content/biorxiv/early/2021/11/04/2021.11.04.467009/F9.large.jpg)

#### Optimal Planting Dates

Here is a plot for optimal planting dates estimated for each year under current and future climate projections in Gosan and Chuncheon.

![](https://www.biorxiv.org/content/biorxiv/early/2021/11/04/2021.11.04.467009/F10.large.jpg)

## Extras

####  Exercise: Add interpretation and discussion to your responses to all questions as appropriate.

#### Exercise (extra): Run the garlic model with Seattle weather data

#### Exercise (extra): Evaluate if and how the optimal planting date changes in Seattle area if atmospheric [CO2] is doubled and air temperatures rise uniformly by 3C.